# Entraînement du Modèle Smart Recycle avec PyTorch & MPS

Ce notebook permet d'entraîner un modèle de classification d'images (ResNet18) sur le dataset TrashNet.
Il est optimisé pour les Mac Apple Silicon (M1/M2/M3) en utilisant l'accélération MPS (Metal Performance Shaders).

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy

# Vérification du device (MPS pour Mac, CUDA pour NVIDIA, CPU sinon)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Utilisation du device : {device}")

Utilisation du device : mps


## 1. Préparation des Données

In [2]:
# Transformations : Redimensionnement, Normalisation (ImageNet stats)
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../data'  # On suppose que le notebook est dans 01_IA_LAB/notebooks/
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(f"Classes : {class_names}")
print(f"Images Train : {dataset_sizes['train']}")
print(f"Images Val : {dataset_sizes['val']}")

Classes : ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Images Train : 2019
Images Val : 508


## 2. Chargement du Modèle (Transfer Learning)

In [3]:
model_ft = models.resnet18(pretrained=True)

# On gèle les poids (optionnel, selon stratégie)
# for param in model_ft.parameters():
#     param.requires_grad = False

# Remplacement de la dernière couche fully connected
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/Users/ramadane/Documents/GitHub/smart-recycle-project/venv/lib/python3.14/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ramadane/Documents/GitHub/smart-recycle-project/venv/lib/python3.14/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 3. Boucle d'Entraînement

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.float() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 1.2226 Acc: 0.5423
val Loss: 0.7145 Acc: 0.7618

Epoch 1/9
----------
train Loss: 0.7071 Acc: 0.7514
val Loss: 0.4948 Acc: 0.8327

Epoch 2/9
----------
train Loss: 0.5644 Acc: 0.8128
val Loss: 0.3939 Acc: 0.8701

Epoch 3/9
----------
train Loss: 0.5117 Acc: 0.8227
val Loss: 0.3885 Acc: 0.8858

Epoch 4/9
----------
train Loss: 0.3941 Acc: 0.8648
val Loss: 0.3552 Acc: 0.8681

Epoch 5/9
----------
train Loss: 0.3605 Acc: 0.8722
val Loss: 0.3387 Acc: 0.8976

Epoch 6/9
----------
train Loss: 0.3404 Acc: 0.8836
val Loss: 0.3322 Acc: 0.8819

Epoch 7/9
----------
train Loss: 0.2837 Acc: 0.9118
val Loss: 0.3123 Acc: 0.9055

Epoch 8/9
----------
train Loss: 0.2980 Acc: 0.9009
val Loss: 0.3169 Acc: 0.8976

Epoch 9/9
----------
train Loss: 0.2737 Acc: 0.9059
val Loss: 0.3038 Acc: 0.8996

Training complete in 9m 45s
Best val Acc: 0.905512


## 4. Sauvegarde du Modèle

In [6]:
import os

save_path = '../models/waste_model.pth'
torch.save(model_ft.state_dict(), save_path)
print(f"Modèle sauvegardé dans {save_path}")

Modèle sauvegardé dans ../models/waste_model.pth
